In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
                 'rdm_padding', 'aug_padding']
list_padding_short = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
                 'rdm_padding']
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)

batch_size = 54
epochss = 200


### Loading stuff

In [3]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

In [4]:
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_2 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 76)                0         
__________

In [ ]:
task = "task2/"
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else:
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        folder_cp = ''.join(string for string in [folder, task, model_type, '/', str(idx)]) 
        #print(folder_cp)
        if not os.path.exists(os.path.join(absPath, 'data/checkpoint/', folder_cp)):
            os.makedirs(''.join(string for string in [absPath, 'data/checkpoint/', folder_cp]))

        callbacks_list = calling_callbacks(folder_cp, folder, model_type, None, None, None, 
                                   None, False, False, False, False)
        folder_task1 =  ''.join(string for string in [folder, task])
        model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
        #writing log file 
        log_file = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, 'log_file.txt' ]) 
        f = open(log_file, 'a+')
        print('Model type: %s \n' % model_type, file=f)
        print('Fold: %i \n' % idx, file=f)
 
        start = time.time()
        formatted_time = datetime.datetime.now()
        print('Starting time: %s \n' % formatted_time, file=f)
        history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len_train/batch_size),
                              validation_steps=int(len_val/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
        end = time.time()
        formatted_endtime = datetime.datetime.now()
        print('Finishing time: %s \n' % formatted_endtime, file=f)
        count_time(start, end, folder, model_type)
        saving_results(history, model_type, folder, idx, True)
        f.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_5 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 76)                0         
__________

138/138 [==============================] - 2s 15ms/step - loss: 0.4871 - acc: 0.2525 - val_loss: 0.1367 - val_acc: 0.1737

Epoch 00030: val_acc did not improve from 0.20179
Epoch 31/200
138/138 [==============================] - 2s 14ms/step - loss: 0.4936 - acc: 0.2609 - val_loss: 0.1329 - val_acc: 0.1935

Epoch 00031: val_acc did not improve from 0.20179
Epoch 32/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4889 - acc: 0.2564 - val_loss: 0.1444 - val_acc: 0.1731

Epoch 00032: val_acc did not improve from 0.20179
Epoch 33/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4798 - acc: 0.2594 - val_loss: 0.1334 - val_acc: 0.2229

Epoch 00033: val_acc improved from 0.20179 to 0.22286, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/post_padding/0/weights-improvement-033-0.2229.hdf5
Epoch 34/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4738 - acc: 0.2728 - val_loss: 0.

138/138 [==============================] - 2s 15ms/step - loss: 0.3401 - acc: 0.3392 - val_loss: 0.0827 - val_acc: 0.2088

Epoch 00069: val_acc did not improve from 0.28991
Epoch 70/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3338 - acc: 0.3407 - val_loss: 0.0952 - val_acc: 0.2139

Epoch 00070: val_acc did not improve from 0.28991
Epoch 71/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3157 - acc: 0.3457 - val_loss: 0.0809 - val_acc: 0.2114

Epoch 00071: val_acc did not improve from 0.28991
Epoch 72/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3515 - acc: 0.3438 - val_loss: 0.0889 - val_acc: 0.2158

Epoch 00072: val_acc did not improve from 0.28991
Epoch 73/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3147 - acc: 0.3468 - val_loss: 0.0740 - val_acc: 0.2433

Epoch 00073: val_acc did not improve from 0.28991
Epoch 74/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3110

138/138 [==============================] - 2s 15ms/step - loss: 0.2567 - acc: 0.3775 - val_loss: 0.0564 - val_acc: 0.2407

Epoch 00113: val_acc did not improve from 0.28991
Epoch 114/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2509 - acc: 0.3862 - val_loss: 0.0662 - val_acc: 0.2324

Epoch 00114: val_acc did not improve from 0.28991
Epoch 115/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2483 - acc: 0.3874 - val_loss: 0.0532 - val_acc: 0.2510

Epoch 00115: val_acc did not improve from 0.28991
Epoch 116/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2497 - acc: 0.3873 - val_loss: 0.0529 - val_acc: 0.2152

Epoch 00116: val_acc did not improve from 0.28991
Epoch 117/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2416 - acc: 0.3862 - val_loss: 0.0686 - val_acc: 0.2299

Epoch 00117: val_acc did not improve from 0.28991
Epoch 118/200
138/138 [==============================] - 2s 15ms/step - loss: 0

138/138 [==============================] - 2s 15ms/step - loss: 0.2339 - acc: 0.4004 - val_loss: 0.0656 - val_acc: 0.1928

Epoch 00157: val_acc did not improve from 0.28991
Epoch 158/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2322 - acc: 0.3983 - val_loss: 0.0479 - val_acc: 0.2433

Epoch 00158: val_acc did not improve from 0.28991
Epoch 159/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2142 - acc: 0.4014 - val_loss: 0.0506 - val_acc: 0.1973

Epoch 00159: val_acc did not improve from 0.28991
Epoch 160/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2271 - acc: 0.3986 - val_loss: 0.0456 - val_acc: 0.2056

Epoch 00160: val_acc did not improve from 0.28991
Epoch 161/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2260 - acc: 0.3937 - val_loss: 0.0465 - val_acc: 0.2101

Epoch 00161: val_acc did not improve from 0.28991
Epoch 162/200
138/138 [==============================] - 2s 15ms/step - loss: 0

...

138/138 [==============================] - 2s 15ms/step - loss: 0.2534 - acc: 0.3925 - val_loss: 0.0396 - val_acc: 0.1992

Epoch 00160: val_acc did not improve from 0.24457
Epoch 161/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2580 - acc: 0.3940 - val_loss: 0.0460 - val_acc: 0.2043

Epoch 00161: val_acc did not improve from 0.24457
Epoch 162/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2623 - acc: 0.3897 - val_loss: 0.0453 - val_acc: 0.1999

Epoch 00162: val_acc did not improve from 0.24457
Epoch 163/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2635 - acc: 0.3908 - val_loss: 0.0473 - val_acc: 0.1973

Epoch 00163: val_acc did not improve from 0.24457
Epoch 164/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2733 - acc: 0.3867 - val_loss: 0.0340 - val_acc: 0.2069

Epoch 00164: val_acc did not improve from 0.24457
Epoch 165/200
138/138 [==============================] - 2s 15ms/step - loss: 0

Epoch 1/200
138/138 [==============================] - 2s 18ms/step - loss: 1.2672 - acc: 0.1520 - val_loss: 0.3132 - val_acc: 0.1009

Epoch 00001: val_acc improved from -inf to 0.10089, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/post_padding/3/weights-improvement-001-0.1009.hdf5
Epoch 2/200
138/138 [==============================] - 2s 15ms/step - loss: 0.8239 - acc: 0.1542 - val_loss: 0.3026 - val_acc: 0.1003

Epoch 00002: val_acc did not improve from 0.10089
Epoch 3/200
138/138 [==============================] - 2s 15ms/step - loss: 0.8028 - acc: 0.1669 - val_loss: 0.2953 - val_acc: 0.1022

Epoch 00003: val_acc improved from 0.10089 to 0.10217, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/post_padding/3/weights-improvement-003-0.1022.hdf5
Epoch 4/200
138/138 [==============================] - 2s 15ms/step - loss: 0.7825 - acc: 0.1773 - val_loss: 0.2810 - val_acc: 0.1015



Epoch 00036: val_acc did not improve from 0.23372
Epoch 37/200
138/138 [==============================] - 2s 15ms/step - loss: 0.5070 - acc: 0.2589 - val_loss: 0.1801 - val_acc: 0.1858

Epoch 00037: val_acc did not improve from 0.23372
Epoch 38/200
138/138 [==============================] - 2s 15ms/step - loss: 0.5044 - acc: 0.2597 - val_loss: 0.1743 - val_acc: 0.2490

Epoch 00038: val_acc improved from 0.23372 to 0.24904, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/post_padding/3/weights-improvement-038-0.2490.hdf5
Epoch 39/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4969 - acc: 0.2646 - val_loss: 0.1698 - val_acc: 0.1980

Epoch 00039: val_acc did not improve from 0.24904
Epoch 40/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4945 - acc: 0.2617 - val_loss: 0.1637 - val_acc: 0.2133

Epoch 00040: val_acc did not improve from 0.24904
Epoch 41/200
138/138 [==========================

138/138 [==============================] - 2s 15ms/step - loss: 0.4327 - acc: 0.2931 - val_loss: 0.1413 - val_acc: 0.1967

Epoch 00080: val_acc did not improve from 0.24904
Epoch 81/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4423 - acc: 0.2858 - val_loss: 0.1494 - val_acc: 0.1654

Epoch 00081: val_acc did not improve from 0.24904
Epoch 82/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4391 - acc: 0.2921 - val_loss: 0.1487 - val_acc: 0.1648

Epoch 00082: val_acc did not improve from 0.24904
Epoch 83/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4353 - acc: 0.2923 - val_loss: 0.1740 - val_acc: 0.1213

Epoch 00083: val_acc did not improve from 0.24904
Epoch 84/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4371 - acc: 0.2876 - val_loss: 0.1426 - val_acc: 0.1584

Epoch 00084: val_acc did not improve from 0.24904
Epoch 85/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4515

138/138 [==============================] - 2s 15ms/step - loss: 0.4090 - acc: 0.3119 - val_loss: 0.1354 - val_acc: 0.1705

Epoch 00124: val_acc did not improve from 0.24904
Epoch 125/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3994 - acc: 0.3167 - val_loss: 0.1056 - val_acc: 0.1928

Epoch 00125: val_acc did not improve from 0.24904
Epoch 126/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4030 - acc: 0.3166 - val_loss: 0.1128 - val_acc: 0.2056

Epoch 00126: val_acc did not improve from 0.24904
Epoch 127/200
 78/138 [===============>..............] - ETA: 0s - loss: 0.3025 - acc: 0.2493